# API data for premier league current season

## Set environment

In [1]:
import os
from pathlib import Path
import numpy as np
import pandas as pd
pd.set_option('display.max_columns', 500)
from datetime import datetime

from dotenv import load_dotenv

import xlsxwriter

import requests
import json

# current date
date = datetime.now().strftime("%Y%m%d-%H%M")
print(date) 

20230123-1758


In [12]:
#Set the current environment
# env = 'development'

# #Load the appropriate .env file
# if env == 'development':
load_dotenv('../../.env')
# elif env == 'production':
#     load_dotenv('.env.prod')

True

### CSV export function

In [3]:
# create function to export to csv
def export_csv(df, filename):
    df.to_csv(f'../../output/{date}-{filename}.csv', encoding='utf-8', index=False)
    print(f"Exported {filename}")

### Excel Export function

In [4]:
# create function to export to excel as a table
def export_excel(df, sheet_name):
    # Create a Pandas Excel writer using XlsxWriter as the engine.
    writer = pd.ExcelWriter(f'../../output/{date}-output.xlsx', engine='xlsxwriter')
    # Convert the dataframe to an XlsxWriter Excel object.
    df.to_excel(writer, sheet_name=sheet_name)
    # Get the xlsxwriter workbook and worksheet objects.
    workbook = writer.book
    worksheet = writer.sheets[sheet_name]
    # format as a table
    worksheet.add_table(0, 0, len(df), len(df.columns)-1, {'columns': [{'header': column} for column in df.columns]})
    writer.save()


In [13]:
# Connect to SQL database with SqlAlchemy  
import pyodbc
import sqlalchemy as sql
from sqlalchemy import create_engine

server = os.environ['SERVER']
database = os.environ['DATABASE']
driver = os.environ['DRIVER']

# create connection string
conn_str = f"mssql+pyodbc://{server}/{database}?driver={driver}?trusted_connection=yes"
# create engine
engine = sql.create_engine(conn_str)

# create connection
conn = engine.connect()


InterfaceError: (pyodbc.InterfaceError) ('IM002', '[IM002] [Microsoft][ODBC Driver Manager] Data source name not found and no default driver specified (0) (SQLDriverConnect)')
(Background on this error at: https://sqlalche.me/e/14/rvf5)

## API call

### Key columns to keep

In [14]:
# keep columns
keep_cols = [
    'id',
    'utcDate',
    'status',
    'matchday',
    # 'season.id',
    # 'season.currentMatchday',
    # 'homeTeam.id',
    # 'homeTeam.name',
    'homeTeam.shortName',
    # 'awayTeam.id',
    # 'awayTeam.name',
    'awayTeam.shortName',
    # 'score.winner',
    'score.fullTime.home',
    'score.fullTime.away',
    # 'score.halfTime.home',
    # 'score.halfTime.away'
]

### API data call

Add scoreline to dataframe

In [15]:
# All of premier league history since 1992-08-15
# uri = 'https://api.football-data.org/v2/competitions/PL/matches?dateFrom=2021-08-01'


In [ ]:

api_key = os.environ['API_AUTH_KEY']


In [29]:

uri = 'http://api.football-data.org/v4/competitions/PL/matches?season=2020'
headers = { 'X-Auth-Token':  api_key}

response = requests.get(uri, headers=headers)
data = response.json()
df = pd.json_normalize(data, record_path = ['matches'])
df = df[keep_cols]
df['utcDate'] = pd.to_datetime(df['utcDate'])
df['score.fullTime'] = df['score.fullTime.home'].astype(str) + '-' + df['score.fullTime.away'].astype(str)
print(df.shape)
df.head()


(380, 9)


,id,utcDate,status,matchday,homeTeam.shortName,awayTeam.shortName,score.fullTime.home,score.fullTime.away,score.fullTime
0,303759,2020-09-12 11:30:00+00:00,FINISHED,1,Fulham,Arsenal,0,3,0-3
1,303764,2020-09-12 14:00:00+00:00,FINISHED,1,Crystal Palace,Southampton,1,0,1-0
2,303760,2020-09-12 16:30:00+00:00,FINISHED,1,Liverpool,Leeds United,4,3,4-3
3,303763,2020-09-12 19:00:00+00:00,FINISHED,1,West Ham,Newcastle,0,2,0-2
4,303762,2020-09-13 13:00:00+00:00,FINISHED,1,West Brom,Leicester City,0,3,0-3


## Pivot results 

In [17]:
# pivot
df_table = df.pivot_table(index=['homeTeam.shortName'], columns=['awayTeam.shortName'], fill_value='.', values=['score.fullTime'], aggfunc='first')

df_table.head()

score.fullTime                                    \
awayTeam.shortName        Arsenal Aston Villa Bournemouth Brentford   
homeTeam.shortName                                                    
Arsenal                         .         2-1           .         .   
Aston Villa                     .           .           .       4-0   
Bournemouth                   0-3         2-0           .       0-0   
Brentford                     0-3           .         2-0         .   
Brighton Hove                 2-4         1-2           .         .   

                                                                        \
awayTeam.shortName Brighton Hove Chelsea Crystal Palace Everton Fulham   
homeTeam.shortName                                                       
Arsenal                        .       .              .       .    2-1   
Aston Villa                    .     0-2              .     2-1      .   
Bournemouth                    .       .            0-2     3-0      .   
Brentford                    2-0     0-0              .     1-1      .   
Brighton Hove                  .     4-1              .       .      .   

                                                                              \
awayTeam.shortName Leeds United Leicester City Liverpool Man City Man United   
homeTeam.shortName                                                             
Arsenal                       .            4-2       3-2        .        3-2   
Aston Villa                 2-1              .       1-3      1-1        3-1   
Bournemouth                   .            2-1         .        .          .   
Brentford                   5-2              .       3-1        .        4-0   
Brighton Hove               1-0            5-2       3-0        .          .   

                                                                        \
awayTeam.shortName Newcastle Nottingham Southampton Tottenham West Ham   
homeTeam.shortName                                                       
Arsenal                  0-0        5-0           .       3-1      3-1   
Aston Villa                .          .         1-0         .      0-1   
Bournemouth                .        1-1         0-1       2-3        .   
Brentford                  .          .           .       2-2        .   
Brighton Hove            0-0        0-0           .       0-1        .   

                                  
awayTeam.shortName Wolverhampton  
homeTeam.shortName                
Arsenal                        .  
Aston Villa                  1-1  
Bournemouth                  0-0  
Brentford                    1-1  
Brighton Hove                  .

In [18]:
# rename columns awayTeam.shortName to null

# df_table.columns = df_table.columns.droplevel(0)
# df_table.columns.name = None

# reset index
df_table2 = df_table.reset_index()

# drop first row
df_table2.columns = df_table2.columns.droplevel(0)
# rename column 2 to home team
df_table2.rename(columns={ df_table2.columns[0]: "homeTeam"}, inplace = True)
# drop index column



# export to csv using function
export_csv(df_table2, "table")
# export_excel(df_table, "table")


df_table2.head()

Exported table


awayTeam.shortName,homeTeam,Arsenal,Aston Villa,Bournemouth,Brentford,Brighton Hove,Chelsea,Crystal Palace,Everton,Fulham,Leeds United,Leicester City,Liverpool,Man City,Man United,Newcastle,Nottingham,Southampton,Tottenham,West Ham,Wolverhampton
0,Arsenal,.,2-1,.,.,.,.,.,.,2-1,.,4-2,3-2,.,3-2,0-0,5-0,.,3-1,3-1,.
1,Aston Villa,.,.,.,4-0,.,0-2,.,2-1,.,2-1,.,1-3,1-1,3-1,.,.,1-0,.,0-1,1-1
2,Bournemouth,0-3,2-0,.,0-0,.,.,0-2,3-0,.,.,2-1,.,.,.,.,1-1,0-1,2-3,.,0-0
3,Brentford,0-3,.,2-0,.,2-0,0-0,.,1-1,.,5-2,.,3-1,.,4-0,.,.,.,2-2,.,1-1
4,Brighton Hove,2-4,1-2,.,.,.,4-1,.,.,.,1-0,5-2,3-0,.,.,0-0,0-0,.,0-1,.,.


In [21]:
print(df.shape)

(198, 11)


## Create Matches DataFrame with a row for each team 

run function 

In [20]:
points_map = {
    'W': 3,
    'D': 1,
    'L': 0
}

def get_result(score, score_opp):
    if score == score_opp:
        return 'D'
    elif score > score_opp:
        return 'W'
    else:
        return 'L'

drop_cols = ['homeTeam.shortName','awayTeam.shortName', 'score.fullTime.home', 'score.fullTime.away']

# convert each match into two rows (one for each team)

df['H'] = df['homeTeam.shortName']
df['A'] = df['awayTeam.shortName']

df_matches = pd.melt(
    df,
    id_vars=keep_cols,
    value_vars=["H", "A"],
    var_name="home_away",
    value_name="team",
)

print(df_matches.shape)
df_matches.head()


(396, 10)


,id,utcDate,status,matchday,homeTeam.shortName,awayTeam.shortName,score.fullTime.home,score.fullTime.away,home_away,team
0,416384,2022-08-05 19:00:00+00:00,FINISHED,1,Crystal Palace,Arsenal,0,2,H,Crystal Palace
1,416383,2022-08-06 11:30:00+00:00,FINISHED,1,Fulham,Liverpool,2,2,H,Fulham
2,416378,2022-08-06 14:00:00+00:00,FINISHED,1,Tottenham,Southampton,4,1,H,Tottenham
3,416379,2022-08-06 14:00:00+00:00,FINISHED,1,Newcastle,Nottingham,2,0,H,Newcastle
4,416381,2022-08-06 14:00:00+00:00,FINISHED,1,Leeds United,Wolverhampton,2,1,H,Leeds United


In [ ]:
df_matches['opponent'] = np.where(
    df_matches['home_away'] == 'H', 
    df_matches['awayTeam.shortName'], 
    df_matches['homeTeam.shortName']
    )

# full time goals
df_matches["goals_scored"] = np.where(
    df_matches["team"] == df_matches["homeTeam.shortName"],
    df_matches["score.fullTime.home"],
    df_matches["score.fullTime.away"],
)
df_matches["goals_against"] = np.where(
    df_matches["team"] != df_matches["homeTeam.shortName"],
    df_matches["score.fullTime.home"],
    df_matches["score.fullTime.away"],
)
df_matches["result"] = np.vectorize(get_result)(
    df_matches["goals_scored"], df_matches["goals_against"]
)
df_matches["points"] = df_matches["result"].map(points_map)


# sort df_matches by id
df_matches = (df_matches
    .drop(drop_cols, axis=1)
    .sort_values(by=['utcDate', 'id'], ascending=True)
    .reset_index(drop=True)
    )

# export to csv using function
export_csv(df_matches, "matches")
# export_excel(df_matches, "matches")

df_matches.head()

## Create league table

In [ ]:
def league(x):
    results = {
        'played': x['result'].count(),
        'wins': (x['result'] == 'W').sum(),
        'draws': (x['result'] == 'D').sum(),
        'losses': (x['result'] == 'L').sum(),
        'goalsFor': x['goals_scored'].sum(),
        'goalsAgainst': x['goals_against'].sum(),
        'goalDiff': x['goals_scored'].sum() - x['goals_against'].sum(),
        'points': x['points'].sum(),
        'form': ''.join(x['result'].tail(5).tolist()),
        # win %
        'win%': '{:.0%}'.format((x['result'] == 'W').sum() / x['result'].count()),        
        # percentage of points from last 5 games
        'form%': '{:.0%}'.format(x['points'].tail(5).sum() / 15),       
        # goals per game
        'goalsScoredPg': '{:.2f}'.format(x['goals_scored'].sum() / x['result'].count()),
        # goals against per game
        'goalsAgainstPg': '{:.2f}'.format(x['goals_against'].sum() / x['result'].count()),
        # goal difference per game
        'goalDifferencePg': '{:.2f}'.format((x['goals_scored'].sum() - x['goals_against'].sum()) / x['result'].count()),
        # points per game
        'pointsPg': '{:.2f}'.format(x['points'].sum() / x['result'].count()),
        # maximum possible points remainig game for the seaon * 3
        'maxPoints': ((38 - x['result'].count()) * 3) + x['points'].sum(),
        # Expected points, based on points per game
        'expectedPoints': '{:.0f}'.format((x['points'].sum() / x['result'].count()) * (38 - x['result'].count()) + x['points'].sum()),
    }
    return pd.Series(results)

# apply league function
df_league = df_matches.groupby('team').apply(league)
# merge in form
# df_league = df_league.merge(df_form, left_index=True, right_index=True)
# Sort by points and then goal difference
df_league = df_league.sort_values(by=['points', 'goalDiff'], ascending=False)
# add column for position
df_league['position'] = df_league['points'].rank(ascending=False, method='first')
# reformat position to int
df_league['position'] = df_league['position'].astype(int)

# Previous weeks league table
def leaguePrev(x):
    results = {
        'pointsPrev': x['points'].sum(),
        'goalDiffPrev': x['goals_scored'].sum() - x['goals_against'].sum(),
    }
    return pd.Series(results)

# df_matchesPrev where matchday does not equal max
df_matchesPrev = df_matches[df_matches['matchday'] != df_matches['matchday'].max()]

# apply league_prev function
df_leaguePrev = df_matchesPrev.groupby('team').apply(leaguePrev)
# sort by points and then goal difference
df_leaguePrev = df_leaguePrev.sort_values(by=['pointsPrev', 'goalDiffPrev'], ascending=False)
# add column for position
df_leaguePrev['positionPrev'] = df_leaguePrev['pointsPrev'].rank(ascending=False, method='first')
# reformat position to int
df_leaguePrev['positionPrev'] = df_leaguePrev['positionPrev'].astype(int)

# join df_league with df_leaguePrev on index
df_league = df_league.join(df_leaguePrev, how='left')
# loop to calculate position 
df_league['positionChange'] = np.where(
    df_league['position'] > df_league['positionPrev'],
    '▼',
    np.where(
        df_league['position'] < df_league['positionPrev'],
        '▲',
        '=',
    ))
# drop previous position
df_league = df_league.drop(['pointsPrev', 'goalDiffPrev', 'positionPrev'], axis=1)

# export to csv using function
export_csv(df_league, "league")

# print head
df_league.head(20)


Exported league


,played,wins,draws,losses,goalsFor,goalsAgainst,goalDiff,points,form,win%,form%,goalsScoredPg,goalsAgainstPg,goalDifferencePg,pointsPg,maxPoints,expectedPoints,position,positionChange
team,,,,,,,,,,,,,,,,,,,
Arsenal,18,15,2,1,42,14,28,47,WWWDW,83%,87%,2.33,0.78,1.56,2.61,107,99,1,=
Man City,18,12,3,3,46,18,28,39,LWDWL,67%,47%,2.56,1.00,1.56,2.17,99,82,2,=
Newcastle,19,10,8,1,33,11,22,38,WWDDW,53%,73%,1.74,0.58,1.16,2.00,95,76,3,=
Man United,18,12,2,4,29,21,8,38,WWWWW,67%,100%,1.61,1.17,0.44,2.11,98,80,4,=
Tottenham,19,10,3,6,37,27,10,33,WDLWL,53%,47%,1.95,1.42,0.53,1.74,90,66,5,=
Fulham,20,9,4,7,32,29,3,31,WWWWL,45%,80%,1.60,1.45,0.15,1.55,85,59,6,=
Brighton Hove,18,9,3,6,35,25,10,30,LWLWW,50%,60%,1.94,1.39,0.56,1.67,90,63,7,▲
Brentford,19,7,8,4,32,28,4,29,WDWWW,37%,87%,1.68,1.47,0.21,1.53,86,58,8,▲
Liverpool,18,8,4,6,34,25,9,28,WWWLL,44%,60%,1.89,1.39,0.50,1.56,88,59,9,▼


## Form by match day

In [ ]:
# pivot df_match to show team results be match day
df_matchday = df_matches.pivot_table(index=['team'], columns=['matchday'], fill_value='.', values=['result'], aggfunc='first')

df_matchday.head(20)

result                                                         
matchday           1  2  3  4  5  6  7  8  9  10 11 12 13 14 15 16 17 18 19 20
team                                                                          
Arsenal             W  W  W  W  W  L  .  W  W  W  W  .  D  W  W  W  W  W  D  W
Aston Villa         L  W  L  L  L  D  .  W  D  D  L  L  W  L  W  W  L  W  D  W
Bournemouth         W  L  L  L  D  W  .  D  D  W  D  L  L  L  L  W  L  L  L  L
Brentford           D  W  L  D  D  W  .  L  D  L  W  D  L  D  D  W  D  W  W  W
Brighton Hove       W  D  W  W  L  W  .  .  D  L  L  D  L  W  W  L  W  L  W  W
Chelsea             W  D  L  W  L  W  L  .  W  W  W  D  D  L  L  L  W  D  L  W
Crystal Palace      L  D  W  L  D  D  .  .  L  W  D  W  L  W  W  L  L  W  L  L
Everton             L  L  D  D  D  D  .  W  W  L  L  L  W  D  L  L  L  D  L  L
Fulham              D  D  W  L  W  L  W  W  L  L  D  W  W  D  L  L  W  W  W  L
Leeds United        W  D  W  L  D  L  .  .  D  L  L  L  L  W  W  L  L  D  D  L
Leicester City      D  L  L  L  L  L  .  L  W  L  D  W  W  L  W  W  L  L  L  L
Liverpool           D  D  L  W  W  D  .  .  D  L  W  W  L  L  W  W  W  W  L  L
Man City            W  W  D  W  W  D  .  W  W  W  L  .  W  W  W  L  W  D  W  L
Man United          L  L  W  W  W  W  .  .  L  W  D  W  D  W  L  W  W  W  W  W
Newcastle           W  D  D  D  L  D  .  D  W  W  D  W  W  W  W  W  W  D  D  W
Nottingham          L  W  D  L  L  L  .  L  L  D  L  D  W  L  D  W  L  D  W  W
Southampton         L  D  W  L  W  L  .  L  L  L  D  W  D  L  L  L  L  L  L  W
Tottenham           W  D  W  W  D  W  .  W  L  W  W  L  L  W  L  W  D  L  W  L
West Ham            L  L  L  W  D  L  .  L  W  W  D  L  W  L  L  L  L  L  D  L
Wolverhampton       L  D  L  D  D  W  .  L  L  L  W  L  L  D  L  L  W  L  D  W

In [ ]:
# df to show newcastle away and home results for each team from df_matchday
df_newcastle = df_matchday.loc['Newcastle', ['result']]



df_newcastle.head(30)

        matchday
result  1           W
        2           D
        3           D
        4           D
        5           L
        6           D
        7           .
        8           D
        9           W
        10          W
        11          D
        12          W
        13          W
        14          W
        15          W
        16          W
        17          W
        18          D
        19          D
        20          W
Name: Newcastle, dtype: object